<a href="https://colab.research.google.com/github/ijnuegnaj/RAG_PRACTICE/blob/practice7/pandas_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai langchain langchain_community

In [ ]:
import os, openai
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('ssu')

In [ ]:
# 대화형 프롬프트 생성
from langchain.prompts.chat import ChatPromptTemplate

# 모델의 출력을 분석하여 변환
from langchain.schema import BaseOutputParser

# 채팅
from langchain.chat_models import ChatOpenAI

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system' , '''Act as a classifier that accurately categorizes the sentiment of comments.
    Given a user-input comment, write '1' if the comment is positive,
    and '0' if the comment is negative.
    Output the integer '1' or '0' only, without any other text.'''),
    ('human', '{input_text}')
])

#모델의 0, 1 이라는 답변을 내가 원하는 형태로 변경할 수 있게 아웃풋 파서를 상속받아서 새 클래스를 정의
class ClassifierOutputParser(BaseOutputParser):
  #parse 함수는 이전 코드에서 보았던 StructuredOutputParse.parse() 함수를
  #덮어쓰기(오버라이딩)하는 함수
  def parse(self, text:str):
    if '1' in text:
      return 1
    else:
      return 0

In [ ]:
chat = ChatOpenAI()
chain = chat_prompt_template | chat | ClassifierOutputParser()

In [ ]:
#하나의 개별 리부에 대한 chain 실행을 위해서
#chain 실행 -> 답변 받기 까지를 하나의 함수로 묶어준다
def langchain_llm(input_text):
  output = chain.invoke({'input_text': input_text})
  return output

In [ ]:
import urllib.request


urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)


('tarr_train.txt', <http.client.HTTPMessage at 0x7bc73a3fd5d0>)

In [ ]:
import pandas as pd

df = pd.read_csv('./tarr_train.txt', sep='\t')
df.head()

,id,comment,label
0,1,여기 음식은 언제 와도 실망시키지 않아요. 최고!,1
1,2,여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.,1
2,3,"진짜 깔끔하고, 맛도 좋았어요. 추천합니다!",1
3,4,왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음,0
4,5,인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!,1


In [ ]:
#머신러닝 라이브러리 중 scikit-learn > confusion matrix(혼동행렬)
from sklearn.metrics import accuracy_score, confusion_matrix


actual_label = []
predicted_label = []


total = len(df)


#데이터프레임의 각 행마다 함수를 실행할 수 있게
#데이터프레임.iterrows() : 데이터프레임의 각 행을 순회함
for index, row in df.iterrows():
  comment = row['comment']
  actual = row['label'] #실제 긍/부정 라벨링


  predicted = langchain_llm(comment) #llm이 예측한 긍/부정 라벨


  #정확도 점수 판단 '리스트'를 요구
  actual_label.append(actual)
  predicted_label.append(predicted)


  print(f'{index+1}/{total}')
  print(f'comment : {comment}, => predict {predicted}/{actual}')


  if index > 3:
    break

1/300
comment : 여기 음식은 언제 와도 실망시키지 않아요. 최고!, => predict 1/1
2/300
comment : 여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다., => predict 1/1
3/300
comment : 진짜 깔끔하고, 맛도 좋았어요. 추천합니다!, => predict 1/1
4/300
comment : 왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음, => predict 0/0
5/300
comment : 인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!, => predict 1/1


In [ ]:
accuracy = accuracy_score(actual_label, predicted_label)
print(f'accuracy : {accuracy*100 :.2f}')

accuracy : 100.00


In [ ]:
cm = confusion_matrix(actual_label, predicted_label)
print(cm)

[[1 0]
 [0 4]]


##2. 댓글 생성 테스크


In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ('system', '''리뷰를 받은 음식점 사장님이라고 생각하고,
    사용자들의 댓글에 친절하게 반응하세요.
    매장에서 가장 인기있는 메뉴를 (가상으로 만들어서) 은근슬쩍 추천해주세요.'''),
    ('human',"{input_text}")
])

In [ ]:
from langchain.schema.output_parser import StrOutputParser

chat = ChatOpenAI()
chain_generator = chat_prompt_template | chat | StrOutputParser()

def langchain_generator(input_text):
  output= chain_generator.invoke({'input_text':input_text})
  return output

In [ ]:
for index, row in df.iterrows():
  comment = row['comment']

  reply = langchain_generator(comment)

  print(f'{index+1}/{len(df)}')
  print(f'comment : {comment}')
  print(f'reply : {reply}')
  print('--------------------')

  if index > 3:
    break

1/300
comment : 여기 음식은 언제 와도 실망시키지 않아요. 최고!
reply : 안녕하세요! 저희 음식점을 찾아주셔서 감사합니다. 항상 만족해 주셔서 정말 기쁘네요! 만약 다음 방문 때 시도해보실 메뉴를 추천드린다면, 저희의 시그니처 메뉴인 "트러플 크림 파스타"를 권해드리고 싶어요. 부드러운 크림 소스와 트러플 향이 어우러진 이 요리는 많은 분들이 사랑해주시는 인기 메뉴랍니다. 다음에 방문하실 때 한 번 시도해보시면 좋을 것 같아요. 또 다시 방문해주셔서 정말 감사합니다!
--------------------
2/300
comment : 여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
reply : 안녕하세요! 소중한 리뷰 감사드립니다. 라멘이 맛있다니 다행입니다. 저희 매장의 라멘은 진한 국물과 쫄깃한 면으로 손님들께 인기가 많아요. 다음 방문 시에는 저희 추천 메뉴인 '고기고기 라멘'을 한 번 시도해보시는 건 어떨까요? 고기의 풍미가 국물과 면과 어우러져 더욱 깊은 맛을 느낄 수 있을 거에요. 많은 사랑 부탁드립니다. 
--------------------
3/300
comment : 진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
reply : 안녕하세요! 소중한 리뷰 감사드립니다. 깔끔하고 맛있는 음식을 제공하는 데 항상 최선을 다하고 있습니다. 다음에 방문하실 때는 우리 매장의 시그니처 메뉴인 '크림 치즈 파스타'를 한 번 시도해보시는 건 어떨까요? 많은 분들이 사랑하는 인기 메뉴입니다. 또 다른 좋은 경험으로 찾아뵙길 기대하겠습니다. 감사합니다! :)
--------------------
4/300
comment : 왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
reply : 안녕하세요! 소중한 의겢 주셔서 감사합니다. 아쉬워요. 맛에 대한 부분은 개인의 취향에 따라 다를 수 있기 때문에 맛에 대한 피드백은 언제든지 환영이에요. 다음에는 다른 메뉴를 시도해 보시는 건 어떨까요? 예를 들어, 저희 매장에서는 시